## Load packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import squarify

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error

%matplotlib inline
plt.style.use('ggplot')

## Read data 

In [ ]:
#path ='file/'
path = './'
state_ts = pd.read_csv(path+'State_time_series565.csv',parse_dates=['Date'])

## Missing value

In [ ]:
# Analysis
print('Date range:{} to {}'.format(state_ts['Date'].min(),state_ts['Date'].max()))

## Resample

The whole data set is resampled considering month start by mean. We can also resample data by sum,count etc, but resampling by mean will give better result. The sample of data availble for total 257 months.

In [ ]:
state_month = state_ts.resample('MS').mean()
state_month = state_month.reset_index()
state_month.shape





## ARIMA : Auto Regressor Integrated Moving Average


In [ ]:
from statsmodels.tsa.stattools import adfuller,acf,pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import autocorrelation_plot

***
* Convert date column to type datetime[ns] by using pandas built in fuction


In [ ]:
#state_ts['Date'] = pd.datetime(state_ts['Date'])
state_ts = state_ts.set_index('Date')
ts = state_ts['Sold_Price']
ts.head()

***
## Stationary Time series property 

* Time series data is time dependent, sample are taken on constant time interval. We have taken data on every month end.
* Time series will have some form of seasonality trend.
* If time series is stationary will have statistical property such as mean,variance are remains constant over time.
* The covariance of i the term and i+m term should not be a function of time

[Sold Price ](#Sold-Price) plot using plolty is above. 

In [ ]:
plt.figure(figsize=(14,4))
ts.plot()

In [ ]:
# Resample data by monthly
plt.figure(figsize=(14,4))
ts = ts.resample('M').mean()
ts.plot()

In [ ]:
# forward fill for nan values
ts = ts.ffill()

## Check Stationarity
* Plotting rolling statistics: we can plot moving average and moving variance 
* Dickey Fuller test: It is statistical test to check stationarity,
    1. Null Hypothesis H0: Time series is non stationary
    2. Altenate Hypothesis H1: If test statics < critical value reject H0 

In [ ]:
def test_stationarity(timeseries):
    
    #rolling statics
    rol_mean = timeseries.rolling(window = 12).mean()
    rol_std = timeseries.rolling(window = 12).std()
    
    #plot rolling statistics
    plt.figure(figsize=(14,4))
    plt.plot(ts, color = 'b', label = 'Original')
    plt.plot(rol_mean, color = 'r', label = 'Rolling Mean')
    plt.plot(rol_std, color = 'g', label = 'Rolling Std')
    plt.legend(loc='best')
    
    # Dickey fuller test
    print('Perfom Dickey fuller test')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
test_stationarity(ts)

The rolling mean is increasing continualy, so time series is non stationary
In time series test statistics > critacal value of %5, so time series is non stationary.

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(14,5))
ax1, ax2 = ax.flatten()

ts_log = np.log(ts)
ts_log.plot(ax=ax1, label = 'Log',color = 'r')
ax1.legend(loc = 'best')

ts_ma = ts_log.rolling(12).mean()
ts_ma.plot(ax = ax2, label = 'mean')
ax2.legend(loc = 'best')


In [ ]:
plt.figure(figsize=(14,4))
ts_dif = ts_ma - ts_log
ts_dif = ts_dif.dropna() # fill na
ts_dif.plot()


In [ ]:
test_stationarity(ts_dif)

## Eliminating Trend and seasonality
1. Differencing: taking difference with perticualar time lag
2. Decomposing: Modelling both trend and seasonality and removing them from the model

In [ ]:
# Differencing
ts_log_dif = ts_log - ts_log.shift()
plt.figure(figsize=(14,4))
ts_log_dif.plot()

In [ ]:
ts_log_dif.dropna(inplace = True)
test_stationarity(ts_log_dif)

In [ ]:
# Decomposing
docom = seasonal_decompose(ts_dif)
fig,ax = plt.subplots(3,1,figsize=(14,8))
ax
ax[0].plot(docom.resid,label = 'Residual', color = 'r')
ax[0].legend(loc= 'best')
ax[1].plot(docom.seasonal, label = 'Seasonal', color = 'b')
ax[1].legend(loc = 'best')
ax[2].plot(docom.trend,  label = 'Trend', color = 'b')
ax[2].legend(loc = 'best')

In [ ]:
test_stationarity(docom.resid.dropna())

test statistics < critical value & p-value is 0.002
we are getting constant mean and standard deviation.
***
## Forcasting time series using ARIMA

The ARIMA forcasting for stationary time series is nothing but linear equation(like linear regression). The predictor depend on (p, d, q) of Arima model.

Time series linear equation: x(t) = alpha*x(t-1)*error(t) 
* **ARIMA: Auto Regressor Integrated Moving Average**
    1. Number of AR (Auto regressor) term **(p)**: AR term is lag of dependent variable. If p is 3 then predictor for x(t) will be x(t-1)..x(t-3)
    2. Number of MA (Moving Average) term **(q)**: MA term is lag of forcast error of predictor equation. If q is 3 then error for x(t) will be e(t-1)..e(t-3)
    3. Number of Differences **(d)**: The number of times that the raw observations are differenced, also called the degree of differencing.
    

* To determine p and q we will use two plots
    1. Auto Correlation Function **ACF**: It is a measure of correlation between TS and lagged of TS (q)
    2. Partial Auto Correlation Function **PACF**: This measures the correlation between the TS with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons.(p)

In [ ]:
# ACF
lag_acf = acf(ts_dif,nlags=20)
#PACF
lag_pacf = pacf(ts_dif, nlags=20, method='ols')

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(14,4))
ax1, ax2 = ax.flatten()
ax1.plot(lag_acf)
ax1.axhline(y=0,linestyle='--',color= 'gray')
ax1.axhline(y= - 1.96/np.sqrt(len(ts_dif)), linestyle='--',color= 'gray')
ax1.axhline(y=  1.96/np.sqrt(len(ts_dif)), linestyle='--',color= 'gray')

ax2.plot(lag_pacf,)
ax2.axhline(y=0,linestyle = '--', color = 'gray')
ax2.axhline(y = -1.96/np.sqrt(len(ts_dif)), linestyle = '--', color = 'gray')
ax2.axhline(y = 1.96/np.sqrt(len(ts_dif)), linestyle = '--', color = 'gray')

The dotted lines in confidence interval, this can be used to determine **p** and **q**.
  * p: The lag value where the **PACF** chart crosses upper chart for first time.
  * q: The lag value where **ACF** chart crosses upper chart for first time.
Here p = 5, q = 1, order = (5,1,1)

In [ ]:
model = ARIMA(ts_dif, order = (5,1,1))
model_fit = model.fit(disp=5)

model_fit.save('model_fit.pkl')
print(model_fit.summary())
plt.figure(figsize=(14,4))
plt.plot(ts_dif)
plt.plot(model_fit.fittedvalues,color = 'r')

## Grid search method

In [ ]:
#Evaluate arima model for (p,d,q)
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error

In [ ]:
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

In [ ]:
# Evaluate parameter
p_value = range(0,2)
d_value = range(0,2)
q_value = range(0,2)
evaluate_models(ts,p_value,d_value,q_value,) 